<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/proc_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import math

from bs4 import BeautifulSoup as bs
from bs4 import UnicodeDammit

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
df_indice = pd.read_csv('/content/drive/My Drive/indice_invertido.csv')

In [0]:
df_indice.shape

In [0]:
df_arquivos = pd.read_csv('/content/drive/My Drive/positiveFilesList.csv')

In [0]:
df_arquivos.shape

In [0]:
#pegar consulta e separar as palavras
#ir no posting e pegar os 5 primeiros arquivos
#usar o tf-idf pra fazer o ranking
#retornar o ranking
#avaliar o ranking

In [0]:
q = "vidas secas"

def handlingQuery(query):
  hq = query.split()
  return hq

print(handlingQuery(q))

In [0]:
def search(hq, indice):
  df_search = pd.DataFrame()
  for q in hq:
    for w in df_indice['Terms'].values:
      if w in q:
        df_search = df_search.append(df_indice[df_indice['Terms'] == w], ignore_index=True)
  return df_search

In [0]:
df_s = search(handlingQuery(q),df_indice)
df_s.head()

In [0]:
def docs_list(df):
  postings = list()
  for r in df.index:
    postings.append(df.loc[r,'Posting'])

  for i in range(len(postings)):
    new_postings = postings[i].split(', ')
    new_postings[0] = new_postings[0][1:]
    new_postings[len(new_postings)-1] = new_postings[len(new_postings)-1][:-1]
    postings[i] = new_postings

  p = list()
  for i in range(len(postings)):
    for j in range(len(postings[i])):
      p.append(postings[i][j])

  p = list(dict.fromkeys(p))
  p.sort()
  return p

In [0]:
dl = docs_list(df_s)
print(dl)

In [0]:
df_freq = pd.DataFrame(columns=dl)
df_freq = pd.concat([df_s['Terms'], df_s['Freq'], df_freq], axis=1)

for r in df_freq.index:
  for c in range(2, len(df_freq.columns)):
    df_freq.iloc[r, c] = 0

df_freq.iloc[:, 1:] = df_freq.iloc[:, 1:].astype(float)
df_freq.head()

In [0]:
symbolsToReplace = [
    '(', ')', '!', '?', ':', ';', '*', '.', ',', '★', '|', '+', '[', ']', '{', '}', '/', 'ª', 'º', '°', '-',
    '%', '—', '@', '#',  '\"','\'', '<', '>', '=', '´', '`', '“', '$', '&', '’', '¡', '€', 'µ', '¦',
    '\\', '®', '™', '”', '…', '‘', '•', '😍','😙', '❤', '–', '⭐️', '_', '️⭐', '😉', '👏', '¹', '²', '³',
    "£", '¢', '¬', '§', '~', '^', '×', '÷',
    '\u200c', '\u200f', '\u200e', '\t', '\xa0', '\x03', '\u0301', '\u0303'
]

allowedSmallTokens = [
  'fio', 'box', 'god', 'key', 'bit', 'led', 'faq', 'dog', 'sim', 'ovo', 'pão', 'aba', 'new'
]

In [0]:
def getAndCleanHTML(filepath):
  
  page = open(filepath, "rb")

  page = page.read()

  clearPage = UnicodeDammit.detwingle(page)
  
  doc = bs(clearPage, 'lxml')
  
  for script in doc(["script", "style"]):
    script.extract()
  
  docText = doc.get_text(' ')
  
  docText = docText.translate({ord(ch): None for ch in '0123456789'})
  
  docText = docText.lower()
  
  docText = docText.replace("e-book", "ebook")

  docText = docText.replace("blu-ray", "bluray")
  
  for symbol in symbolsToReplace:
    docText = docText.replace(symbol, ' ')
  
  docText = docText.replace("  ", ' ')
  
  lines = (line.strip() for line in docText.splitlines())
  
  chunks = (phrase.strip() for line in lines for phrase in line.split(' '))

  text = ' '.join(chunk for chunk in chunks if chunk)
  
  return text

In [0]:
df_arquivos.head(n=10)

In [0]:
queryl = handlingQuery(q)
for i in range(len(dl)):
  doc = getAndCleanHTML(df_arquivos.loc[int(dl[i]),'0'])
  doc = doc.split()
  for w in doc:
    for qw in range(len(queryl)):
      if w in queryl[qw]:
        df_freq.at[qw, dl[i]] += 1.0

In [0]:
df_freq.head()

In [0]:
for r in df_freq.index:
  df_freq.at[r, 'Freq'] = math.log(df_arquivos.shape[0]/df_freq.at[r, 'Freq'],10)
  for c in df_freq.columns:
    df_freq.iloc[r, c] = np.multiply(df_freq.at[r, 'Freq'], df_freq.loc[r, c])

df_freq.head()